# CS109A Final Project: Data Driven March Madness 

**Harvard University**<br>
**Fall 2016**<br>
**Authors: Kurt Bullard and Kendrick Vinar**<br>

# Table of Contents
___________

- [Introduction](#Introduction)
- [Abstract](#Abstract)
- [Project Walkthrough](#walkthrough)
- [Data Manipulation](#manipulation)
- [Data Analysis](#analysis)
- [Results](#results)
- [Acknowledgements](#acknowledgements)
- [Areas for Further Study](#exploration)

# Introduction
------------------

March Madness is the colloquial phrase for the pinnacle event of college basketball, the NCAA Men's Basketball Tournament.  Each year in mid-March, sixty-four teams compete for the college basketball national championship. Unlike the NBA playoffs, in which teams play a best-of-seven series to advance, the national champion of college basketball is crowned via a single-elimination bracket. Hence, the madness -- a single loss can erase the significance of an excellent regular season for a favorite (see Kentucky in 2015), and a deep tournament run for an underdog can earn one a place in March Madness lore for years (see Davidson in 2008). What makes March Madness special is its unpredictability. Every year there are dramatic upsets and tournament runs. Despite millions of brackets submitted each year, there has never been a perfect bracket. 

Each year basketball experts, pundits, and casual fans alike fill out a bracket to predict who will win the 63 games each year. Theories and opinions on crafting the ideal bracket abound. Who do you listen to? We decided to put aside our theories and let the data speak. 


# Abstract 
_______________

We created models to generate March Madness predictions using different machine learning algorithms, including logistic regression, SVM, and random forests. We analyzed data from 2003 through the 2016 season. We used a 7-year window time-series method to create seven years of predictions from 2010 to 2016. Each of those years of predictions are trained on seven years of prior tournament data. Three of those years, 2010-2012, we used to tune our models. We used the last four years from 2013-2016 to test the efficacy of our model. We evaluated the performance of our models with a common scoring system used by ESPN, Yahoo Sports, and CBS Sports. Our model generates probabilities for the likelihood of each team advancing in a given matchup, samples from a binomial distribution over the likelihood of the favorite winning, and advances the team that is most likely to advance according to the draw from the distribution. We tested our models using differing numbers of draws. Our results found that logistic regression outperformed random forests and support vector models. 

[say something about results here]

<a id = 'walkthrough'></a>

# Project Walkthrough

Below is a detailed examination of the process by which we created our models. It explains in much greater detail the individual steps of our project, justifications for our rationale, problems we encountered, and decisions we had to make. This portion is intended to be an accessible explanation of our process and reasoning. The table of contents below can be used to navigate to a topic of interest. The table of contents is divided up into two portions roughly corresponding to the two separate files in which we performed this project. The data manipulation portion focuses on the acquisition and preparation of the data for the project. The data analysis portion focuses on the models and results. 

<Return back to the [homepage](home_page.html).>
_____________________

### Data Manipulation
- [Overview](#overview)
- [Code](#Code)
- [Data Sources](#data_sources)
- [Data Cleaning](#data_cleaning)
- [Variable Creation/Selection](#variable_creation)
- [2016 Data Extraction](#2016_data)
- [Preparing the Data for Analysis](#preparation)

### Data Analysis
- [Overview](#overview_2)
- [Code](#code_2)
- [Goals](#goals)
- [Functional Justification](#functional)
- [Scoring System](#scoring)
- [Binomial Sampling and the Case for Variance](#binomial)



<a id = 'manipulation'></a>

# Data Manipulation
___________

<a id = 'overview'></a>

## Overview

The intent of this portion of the project is to organize the data in a fashion such that it is ready to be used by the various machine learning algorithms we test. This includes importing the data, cleaning the data, creating variables, and reducing variables. All of that occurs here. 

<a id = 'Code'></a>

## Code

The code for this portion can be found [here](data_manipulation.html). 

<a id = 'data_sources'></a>

## Data Sources

We retrieved data from two sources: the [2016 Kaggle March Madness Learning Mania](https://www.kaggle.com/c/march-machine-learning-mania-2016) competition and the [Pomeroy College Basketball Rankings](http://www.kenpom.com). The Kaggle dataset provided us with detailed information for each team’s regular season performance in addition to results from that year’s tournament. The Pomeroy rankings—or KenPom, as they are colloquially known— give additional advanced metrics for each team, including but not limited to offensive efficiency, defensive efficiency, and adjusted tempo. 

We imported nine CSV files from these sources, including information about seeding, and box scores of tournament and regular season games from 2003-2016.

<a id = 'data_cleaning'></a>

## Data Cleaning

While sorting through the data, we realized we needed an easy way to distinguish between data for a Duke team that played in 2005 and a Duke team that played in 2010. For this, we created a unique ID for each team that would identify both the team and the year by combining the team name and the season in which it played. 

We merged the KenPom data into the detailed tournament dataset by choosing the variables we wanted to consider for our model. These included rankings such as offensive and defensive efficiency, advanced metrics that were not included in the Kaggle data. 

<a id = 'variable_creation'></a>

## Variable Creation and Selection

We created a variable for tournament seedings [1, 16] to normalize seed across regions. We did this by scraping the integers from the seed column that also referenced the region from which the bracket came. 

We created new variables with data given us from the Kaggle dataset that we felt would be better predictors. For example, we calculated predictors such as assist ratio, which is calculated as the number of assists in a game divided by the number of made field goals in a game. This predictor functions as a proxy for how team-oriented a team is or how reliant they are on star players. We created other variables in a similar fashion, including offensive and defensive rebound ratio, point differential, and the percentage of shots in the game that were two-point field goals and three-point field goals. 

We also created two variables to quantify performance against other teams in the NCAA tournament. One variable represents wins against other teams in the tournament and another variable represents losses against other teams in the tournament. For a team that beats a tournament ranked $n$ in the tournament, we gave them $1/n$ points. This system weights wins against highly-ranked teams in the tournament much more valuable than wins against lowly-ranked teams. This technique for predictor creation was shown to be effective by John Ezekowitz in his paper, [insert link]. 

[kurt can you say more here about what happens next it’s getting pretty fuzzy for me here]

Once we aggregated all the predictors together, we created a correlation matrix to determine which predictors were correlated with each other. We checked for multicollinearity with variance inflation factor (VIF) and eliminated variables. Below is a heat map visual of the correlation between variables before reducing with VIF. [kurt can you say more here?]

![](pics/all_predictors.png)

Below is a second heat map that shows the correlation between variables after reducing with VIF.

![](pics/selected_predictors.png)

[kurt can you also say something about the interaction terms]

After selecting the desired variables, we created a new data frame to hold our selected predictors. We found that a lot of this data manipulation was very time intensive, so we chose to separate this into a distinct notebook that could be run only when changes were made. At the end of this notebook, we export a new CSV file that serves as the starting point for our data analysis in another notebook.



<a id = '2016_data'></a>

## 2016 Data Extraction

Because the Kaggle competition data was put online for people to predict the 2016 tournament, the 2016 tournament games are not in the database. Because of this, we had to slightly alter our data gathering process for 2016 tournament teams. 

We used the tournament slots database to find which teams were in the tournament, and then compiled the 13 important variables for each team that we transferred to the data analysis notebook.

<a id = 'preparation'></a>

## Preparing the Data for Analysis



<a id = 'analysis'></a>

# Data Analysis
_________________

<a id = 'overview_2'></a>

## Overview

This portion deals primarily with tuning models, training models, and reporting results. Again, there are explanations of our process throughout, however greater technical proficiency is expected. 


<a id = 'code_2'></a>

## Code

The code for this portion can be found [here](data_analysis.html). We highly recommend a reading of this file.


<a id = 'tradeoffs'></a>

## Tradeoffs

One thing we noticed early on in the data analysis process was that there was going to be a tradeoff between the mean and variance of our scores. The NCAA selection committee does well with their seeding. When we always selected the team our model predicted would win, we would get very few upsets. The upsets we did get were usually 8-seed vs. 9-seed games, 7-seed vs. 10-seed games, or 4-seed vs. 5-seed games in the second round. Nothing very controversial. We could pick the "worse" team to win but that would come at a cost to the expected value of the bracket. 

However, what picking the "worse" team does is increase the variance of our results. Sometimes we were wrong when we picked the "worse" team, but sometimes we were right. 

[kurt I'm getting lost can you talk about this?]

<a id = 'goals'></a>

## Goals

Given the tradeoff between mean and variance above, the goal of our model was to give ourselves the best chance of scoring very highly in the tournament bracket. Our rationale is that most tournament brackets and pools reward only very high scoring brackets. Usually the winning bracket receives a sizeable prize while 2nd and 3rd usually receive little or nothing.
A bracket that consistently performs above average but doesn't ever score above the 90th percentile is of little value. 

With that in mind, we sought models that would give us the best chance of doing really well, even if that meant that our model tanked some years. We emphasized these values in our tuning process when selecting model parameters. 

For example, if you play a winner-take-all pool with 30 participants with a 20 dollar buy-in, and your model is terrible nine times of out ten but wins once every ten years, your expected value for each year you play is a net gain of 40 dollars. 

Therefore, we are relatively unconcerned with the fact that our models perform poorly sometimes. We are more concerned with the upper-bound performance near the top 10% of brackets. 

<a id = 'functional'></a>

## Functional Justification

We attempted to write the majority of our code in functional form. This strategy is preferred for several reasons. 

First, many of the operations we wish to perform need to be repeated numerous times. For example, there are many times we wish to reindex a dataframe, clean a dataframe of faulty seedings, and more. Therefore, to save us from writing these lines of code over and over, we used functions.

Second, many of the operations we perform are applied over different machine learning algorithms, different years of training data, and more. To maximize flexibility and ease of use, a functional was preferred. 

Last, a functional approach increased the interpretability of our code. Because we packaged our functions into specific tasks,  we were able to understand better what was happening in our code. As this was a partner project, this made working together as a team much easier. 


<a id = 'scoring'></a>

## Scoring

We evaluated the performance of our models with a common scoring system used by ESPN, Yahoo Sports, and CBS Sports. Under this system, one receives $2^{n-1}$ points for a correct prediction in the $n-th$ round. For example, one receives one point for a correct prediction in the 1st round, four points for a correct prediction in the 3rd round, and thirty-two points for correctly predicting the winner. According to this system, a perfect bracket will score 192 points. While there are other scoring systems used for March Madness, this system is the most common. 

Other scoring systems include adding the seed of your winner to the points for a given round. For example, if you correctly pick a 10-seed to beat a 7-seed in the first round, you get 1 + 10 = 11 points. This kind of scoring system massively changes the incentives of the bracket, and our model would not be appropriate for maximizing your score with this system. Adapting our model such that it can handle different scoring systems was not something that we were able to tackle during this project, however we believe this is an interesting area to extend our project in the future. 


Using ESPN's Tournament Challenge, we were able to record nationwide performance for the last two years. Unfortunately, they have closed the webpages for results before 2015. 

2015: Winner scored 183 points. 147 points is 99th percentile. 116 points is 90th percentile. 98 points is 75th percentile. 82 points is 50th percentile. 68 points is 25th percentile. 

2016: Winner scored 173 points. 123 points is 99th percentile. 92 points is 90th percentile. 81 points is 75th percentile. 67 points is 50th percentile. 56 points is 25th percentile. 

An interesting observation here is the variance in nationwide performance. Across the board, there is a dramatic decrease in scoring in each category from 2015 to 2016. 



<a id = 'standard_model'></a>

## Standard Model



<a id = 'risk'></a>

## Risk

In our search for adding variance into our model, we created a risk parameter that defined the decision boundary for classifying a win. Our hope was that we would see more upsets in our model that would lead to increased variance in our predictions. Ideally, our average score would decrease slightly but our variance would increase such that we achieved, on occasion, high scores in the bracket. 

Normally, our decision boundary, or risk parameter, was 0.5. This meant that if our model predicted favorite had a better than 50% chance of winning the game, we would advance them to the next round. We tested how bracket results varied across different decision boundaries, using 0.4, 0.5, 0.6, and 0.7 as our different risk parameters. 

Below is a bracket from 2015 in which we set the risk parameter is 0.5. 

![](pics/bracket_print.png)

As you can see, using a risk parameter of 0.5 biases toward the favorite. This makes sense since the favorite is favored to win, and we won't predict many games when the underdog is favored to win. 

NOTE: INCLUDE BRACKET WITH RISK PARAMETER OF 0.7

<a id = 'binomial'></a>

## Binomial Sampling and the Case for Variance

After creating our risk parameter, we pivoted towards a different method of adding randomness into our bracket predictions. The problem with the 'risk' parameter was that there really wasn't any addition of randomness, there was just a bias towards a larger number of upsets. 

Our solution to this was to fit a binomial distribution over the likelihood of the favorite advancing and vary the number of draws from the distribution. As the number of draws increases, the more the model comes to approximate our original model which advanced the team the model defined as better every time. For example, suppose our model says team 1 has a 60% chance of defeating team 2. In effect, the binomial distribution allows us to flip a weighted coin to decide who will advance. The weighted coin comes up 'advance' for the favorite 60% of the time and 'advance' for the underdodg 40% of the time. If we set the number of coin flips to one, the favorite advances 60% of the time, and the underdog advances 40% of the time. 

While we could have always used a single draw to determine who advances, we felt that a single draw might cost us tournament points in the long run. Why? Because we assume that the expected value of the favorite going forward in the tournament is greater than the expected value of the underdog going forward. For example, maybe an 8-seed gets hot for a game and knocks of a 1-seed. However, it seems likely that while the 8-seed may be better than the seed it received, there is also an element of luck involved in their win. Therefore, we don't expect them to continue winning many games further on into the tournament. In contrast, on average 1-seeds make much deeper tournament runs, accumulating more points. 

By taking more coin flips, say a best-of-3 or best-of-5 game, we bias the result towards the team we think has a better chance of advancing further in the tournament. This makes sense intuitively. If you know you're going to win a coin flip for five dollars 90% of the time, you want to play that game as often as you can because you know you'll come out ahead. Even if you lose the first game, you'll keep playing. The binomial distribution is similar. By increasing the number of coin flips, it's akin to playing an imaginary best-of-3 or best-of-5 series for the right to advance to the next round. 

Below are summary statistics from test data in 2015 with a different number of coin flips. For each coin flip example, we ran 50 brackets. As the data shows, there is a tradeoff between the mean score and the variance of the flips. In order to score a very high bracket, it makes sense to only use a single flip because you maximize your variance, especially if you are competing in a large pool. However, there is a corresponding decrease in mean score. Therefore, if you are competing with only a few friends, it may make sense to use a greater number of coin flips to increase the mean score and decrease variance. 


![](pics/df_w_results.png)

The following graphs show the distribution of scores for each of the three models with one and three votes, respectively. 



![](pics/result_graphs.png)

As you can see, as the votes increase, the mean slightly increases but the variance decreases.  

<a id = 'results'></a>

# Results
_________

In the end, we chose to go with a logistic regression model with a tuning parameter of C = 0.1. The averaged variable coefficients from the 2010-2015 models are displayed as follows: 

![](pics/coefs.png)

As is accepted in conventional wisdom, the general seed of a team and the team's offensive and defensive strength are strong predictors of a team's success in the tournament. The higher the underdog's seed or the lower the favorite's seed, the more likely it was the favorite won. The favorite's ability to hit the offensive glass was also a strong predictor of success—this could be because that stat encompasses other variables such as height and ability to draw big men away from the basket on offense. In addition, a team's ability to force teams to shoot threes also was a good sign of success. This suggest that teams with good interior defense do well in tournaments. An underdog's reliance on free-throws also was good for the favorite. This could be because underdogs no longer have that adavantage when they play better teams, unlike they did in the regular season if they played in weaker conferences. 

A favorite's tendency to turn the ball over was a strong predictor of failure. This makes sense, since an underdog naturally increases its chances of winning by increasing the amount of possessions it has. 

What we found interesting is that, in our model, the variable that weights a favorite's wins over other tournament teams in the regular season yielded a negative coefficient, suggesting that, holding everything else constant, a team does worse having beaten a strong slate of teams over the course of the season. We hypothesized that this is because teams who have beaten a lot of teams may be overseeded in the tournament, since the selection committe is prone to bias and could overweight "marquee wins" over the course of the season. 

In terms of variables that we included that did not seem significant, tempo and defensive rebounding ranked highest. 

<a id = 'exploration'></a>

# Areas for Further Exploration
_________

There are several directions that we would have liked to extend our project if we had the time. These are listed and described in greater detail below.

1. Adapt model to handle multiple scoring systems
2. Portfolio of brackets 
3. Account for point differential
4. Explore further variable introduction and reduction

First, we created a model that optimizes score based on a conventional scoring system that is consistent across top bracket competition sites, most notably the ESPN Bracket Challenge. One extension would be to change the model such that it could be easily optimized for different scoring systems. 

Second, an interesting extension would be to create a portfolio of brackets to optimize the maximum score from those brackets. Many bracket challenges and pools allow the submission of multiple entries. It would be fascinating to write a function that returned a portfolio of $n$ brackets that give the best chance of scoring as high as possible.

Fourth, there were variables that we thought might contain predictive value that we did not have time or ability to include. Some of these include AP Preseason Poll Points, Vegas futures, and individual player statistics.  

We also considered including the quality of coaching and strength of conference, however [this article](http://harvardsportsanalysis.org/2016/03/for-coaches-previous-experience-in-march-madness-may-not-mean-much/) and [this article](http://harvardsportsanalysis.org/2015/03/conference-bias-in-the-ncaa-tournament/), respectively, suggest there is little value in these predictors. 




<a id = acknowledgements></a>

# Acknowledgements
_____________________

We would like to thank Michael Farrell, our project TF, for his advice, feedback, and availability throughout this process. His direction has been invaluable. 

We would also like to thank Kevin Rader, Pavlos Protopapas, and Weiwei Pan, the instructors of CS109a, for the time they have invested in our growth and learning throughout the semester. We are very grateful!